# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data Load

In [2]:
train_data = pd.read_csv('跨境电商效果广告ROI预测挑战赛公开数据/train.csv')
test_data = pd.read_csv('跨境电商效果广告ROI预测挑战赛公开数据/testA.csv')

In [3]:
train_data.head()

,ad_id,ad_set_id,campaign_id,product_id,datetime,account_id,post_id_emb,post_type,countries,gender,...,cum_bounces,cum_sessions,cum_session_duration,cum_add_cart_num,cum_add_payment_info,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi
0,23848303658498562,23848131439913714,23848131439304807,6fc8e657540e589d6f70a91b783e3f38,2022-10-01 00:00:00,196573582962558,d5f21fd53aca5bc7c6131e1bc8f2fcbd,2.0,"US,GB",0,...,NaN,NaN,NaN,0,2,6,2,51.96,0.0,0.0
1,23848303658498562,23848131439913714,23848131439304807,6fc8e657540e589d6f70a91b783e3f38,2022-10-01 01:00:00,196573582962558,d5f21fd53aca5bc7c6131e1bc8f2fcbd,2.0,"US,GB",0,...,NaN,NaN,NaN,0,2,6,2,51.96,0.0,0.0
2,23848303658498562,23848131439913714,23848131439304807,6fc8e657540e589d6f70a91b783e3f38,2022-10-01 02:00:00,196573582962558,d5f21fd53aca5bc7c6131e1bc8f2fcbd,2.0,"US,GB",0,...,NaN,NaN,NaN,0,2,6,2,51.96,0.0,0.0
3,23848303658498562,23848131439913714,23848131439304807,6fc8e657540e589d6f70a91b783e3f38,2022-10-01 03:00:00,196573582962558,d5f21fd53aca5bc7c6131e1bc8f2fcbd,2.0,"US,GB",0,...,NaN,NaN,NaN,0,2,6,2,51.96,0.0,0.0
4,23848303658498562,23848131439913714,23848131439304807,6fc8e657540e589d6f70a91b783e3f38,2022-10-01 04:00:00,196573582962558,d5f21fd53aca5bc7c6131e1bc8f2fcbd,2.0,"US,GB",0,...,NaN,NaN,NaN,0,2,6,2,51.96,0.0,0.0


In [4]:
len(set(train_data['ad_id'].unique().tolist())&set(test_data['ad_id'].unique().tolist()))

0

# Feature Engineering

In [5]:
train_data['datetime'] = pd.to_datetime(train_data['datetime'])
test_data['datetime'] = pd.to_datetime(test_data['datetime'])
train_data['datetime_hour'] = train_data['datetime'].dt.hour
test_data['datetime_hour'] = test_data['datetime'].dt.hour

train_data.drop('datetime', axis=1, inplace=True)
test_data.drop('datetime', axis=1, inplace=True)

for col in ['ad_id', 'ad_set_id', 'campaign_id', 'product_id', 'account_id', 'post_id_emb', 'post_type', 'countries']:
    lbl = LabelEncoder()
    lbl.fit(list(train_data[col]) + list(test_data[col]))
    train_data[col] = lbl.transform(list(train_data[col]))
    test_data[col] = lbl.transform(list(test_data[col]))

train_data['product_id_roi_mean'] = train_data['product_id'].map(train_data.groupby(['product_id'])['roi'].mean())
test_data['product_id_roi_mean'] = test_data['product_id'].map(train_data.groupby(['product_id'])['roi'].mean())

train_data['account_id_roi_mean'] = train_data['account_id'].map(train_data.groupby(['account_id'])['roi'].mean())
test_data['account_id_roi_mean'] = test_data['account_id'].map(train_data.groupby(['account_id'])['roi'].mean())

train_data['countries_roi_mean'] = train_data['countries'].map(train_data.groupby(['countries'])['roi'].mean())
test_data['countries_roi_mean'] = test_data['countries'].map(train_data.groupby(['countries'])['roi'].mean())

train_data['datetime_hour_roi_mean'] = train_data['datetime_hour'].map(train_data.groupby(['datetime_hour'])['roi'].mean())
test_data['datetime_hour_roi_mean'] = test_data['datetime_hour'].map(train_data.groupby(['datetime_hour'])['roi'].mean())

In [6]:
train_data

,ad_id,ad_set_id,campaign_id,product_id,account_id,post_id_emb,post_type,countries,gender,age_min,...,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi,datetime_hour,product_id_roi_mean,account_id_roi_mean,countries_roi_mean,datetime_hour_roi_mean
0,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,0,1.606082,2.249979,1.563869,1.805425
1,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,1,1.606082,2.249979,1.563869,1.798679
2,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,2,1.606082,2.249979,1.563869,1.802189
3,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,3,1.606082,2.249979,1.563869,1.802148
4,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,4,1.606082,2.249979,1.563869,1.801525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656414,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,8,1.549928,1.502077,1.433427,1.829020
656415,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,9,1.549928,1.502077,1.433427,1.817643
656416,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,10,1.549928,1.502077,1.433427,1.809741
656417,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,11,1.549928,1.502077,1.433427,1.803130


In [7]:
nan_columns = ['cum_bounces','cum_sessions','cum_session_duration','bounces','sessions','session_duration']
train_data = train_data.drop(nan_columns, axis=1)

In [8]:
train_data_ids, dev_data_ids = train_test_split(train_data['ad_id'].unique(), train_size=0.8, random_state=0)

In [9]:
train_data_ids.shape, dev_data_ids.shape

((5732,), (1433,))

In [10]:
dev_data = train_data[train_data['ad_id'].isin(dev_data_ids)]
train_data = train_data[train_data['ad_id'].isin(train_data_ids)]

In [11]:
train_data

,ad_id,ad_set_id,campaign_id,product_id,account_id,post_id_emb,post_type,countries,gender,age_min,...,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi,datetime_hour,product_id_roi_mean,account_id_roi_mean,countries_roi_mean,datetime_hour_roi_mean
0,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,0,1.606082,2.249979,1.563869,1.805425
1,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,1,1.606082,2.249979,1.563869,1.798679
2,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,2,1.606082,2.249979,1.563869,1.802189
3,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,3,1.606082,2.249979,1.563869,1.802148
4,1,0,0,209,1,2249,2,19,0,18,...,6,2,51.96,0.0,0.0,4,1.606082,2.249979,1.563869,1.801525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656414,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,8,1.549928,1.502077,1.433427,1.829020
656415,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,9,1.549928,1.502077,1.433427,1.817643
656416,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,10,1.549928,1.502077,1.433427,1.809741
656417,7364,3792,2416,135,31,1812,3,10,2,25,...,0,0,0.00,0.0,0.0,11,1.549928,1.502077,1.433427,1.803130


In [12]:
dev_data

,ad_id,ad_set_id,campaign_id,product_id,account_id,post_id_emb,post_type,countries,gender,age_min,...,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi,datetime_hour,product_id_roi_mean,account_id_roi_mean,countries_roi_mean,datetime_hour_roi_mean
166,2,0,0,209,1,2249,2,19,0,18,...,0,0,0.0,0.0,1.521178,0,1.606082,2.249979,1.563869,1.805425
167,2,0,0,209,1,2249,2,19,0,18,...,0,0,0.0,0.0,1.521178,1,1.606082,2.249979,1.563869,1.798679
168,2,0,0,209,1,2249,2,19,0,18,...,0,0,0.0,0.0,1.521178,2,1.606082,2.249979,1.563869,1.802189
169,2,0,0,209,1,2249,2,19,0,18,...,0,0,0.0,0.0,1.521178,3,1.606082,2.249979,1.563869,1.802148
170,2,0,0,209,1,2249,2,19,0,18,...,0,0,0.0,0.0,1.521178,4,1.606082,2.249979,1.563869,1.801525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656383,7361,3794,2419,135,31,1608,3,10,2,25,...,1,0,0.0,0.0,2.482316,8,1.549928,1.502077,1.433427,1.829020
656384,7361,3794,2419,135,31,1608,3,10,2,25,...,1,0,0.0,0.0,2.482316,9,1.549928,1.502077,1.433427,1.817643
656385,7361,3794,2419,135,31,1608,3,10,2,25,...,1,0,0.0,0.0,2.482316,10,1.549928,1.502077,1.433427,1.809741
656386,7361,3794,2419,135,31,1608,3,10,2,25,...,1,0,0.0,0.0,2.482316,11,1.549928,1.502077,1.433427,1.803130


In [13]:
feature_columns = train_data.iloc[:].drop('roi', axis=1).columns
feature_columns

Index(['ad_id', 'ad_set_id', 'campaign_id', 'product_id', 'account_id',
       'post_id_emb', 'post_type', 'countries', 'gender', 'age_min', 'age_max',
       'spend', 'impressions', 'reach', 'clicks', 'engagement_nums',
       'post_shares', 'post_reactions', 'post_comments', 'post_saves',
       'watch15s', 'watch30s', 'watch_p25', 'watch_p50', 'watch_p75',
       'watch_p95', 'watch_p100', 'watches', 'add_cart_num',
       'add_payment_info', 'initiates_checkout', 'purchase', 'income_1',
       'income_2', 'cum_spend', 'cum_impressions', 'cum_reach', 'cum_clicks',
       'cum_engagement_nums', 'cum_post_shares', 'cum_post_reactions',
       'cum_post_comments', 'cum_post_saves', 'cum_watch15s', 'cum_watch30s',
       'cum_watch_p25', 'cum_watch_p50', 'cum_watch_p75', 'cum_watch_p95',
       'cum_watch_p100', 'cum_watches', 'cum_add_cart_num',
       'cum_add_payment_info', 'cum_initiates_checkout', 'cum_purchase',
       'cum_income_1', 'cum_income_2', 'datetime_hour', 'product_id_r

# Model

In [14]:
# lightgbm
from lightgbm import LGBMRegressor
model = LGBMRegressor()

model.fit(
    train_data.iloc[:][feature_columns],
    train_data.iloc[:]['roi'], categorical_feature=['ad_id', 'ad_set_id', 'campaign_id', 'product_id', 'account_id', 'post_id_emb', 'post_type', 'countries']
)

dev_data['lgbm_y_pred'] = model.predict(dev_data[feature_columns])
dev_data['lgbm_wmse'] = dev_data.apply(lambda x: (x['datetime_hour']+1) * 1.0 / 13 * abs(x['roi'] - x['lgbm_y_pred']), axis=1)
np.mean(dev_data['lgbm_wmse'])
# df = pd.read_csv('提交示例.csv')
# df['roi'] = model.predict(test_data.iloc[:].drop('uuid', axis=1))
# df.to_csv('submit.csv', index=None)

D:\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


0.819817340110646

In [15]:
# xgboost
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(train_data.iloc[:][feature_columns],
        train_data.iloc[:]['roi']
)
dev_data['xgb_y_pred'] = model.predict(dev_data.iloc[:][feature_columns])
dev_data['xgb_wmse'] = dev_data.apply(lambda x: (x['datetime_hour']+1) * 1.0 / 13 * abs(x['roi'] - x['xgb_y_pred']), axis=1)
np.mean(dev_data['xgb_wmse'])

0.8635157501524727

In [16]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_columns', None)
train_data.describe()

,ad_id,ad_set_id,campaign_id,product_id,account_id,post_id_emb,post_type,countries,gender,age_min,age_max,spend,impressions,reach,clicks,engagement_nums,post_shares,post_reactions,post_comments,post_saves,watch15s,watch30s,watch_p25,watch_p50,watch_p75,watch_p95,watch_p100,watches,add_cart_num,add_payment_info,initiates_checkout,purchase,income_1,income_2,cum_spend,cum_impressions,cum_reach,cum_clicks,cum_engagement_nums,cum_post_shares,cum_post_reactions,cum_post_comments,cum_post_saves,cum_watch15s,cum_watch30s,cum_watch_p25,cum_watch_p50,cum_watch_p75,cum_watch_p95,cum_watch_p100,cum_watches,cum_add_cart_num,cum_add_payment_info,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi,datetime_hour,product_id_roi_mean,account_id_roi_mean,countries_roi_mean,datetime_hour_roi_mean
count,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,5.282540e+05,5.282540e+05,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,5.282540e+05,528254.000000,528254.000000,528254.000000,528254.000000,5.282540e+05,528254.000000,528254.000000,528254.00000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000,528254.000000
mean,2820.942870,1296.693469,817.733463,195.795432,36.483078,1390.733073,2.054824,14.082381,0.338898,26.328626,64.541113,4.057076,200.240769,182.690369,3.376681,6.524053,0.072300,0.392595,0.032371,0.089406,17.621118,9.263317,29.887495,17.305052,11.901517,8.368092,6.934431,135.581518,0.689405,0.070154,0.109180,0.069557,6.782267,2.538947,1948.637683,1.009538e+05,8.574527e+04,1807.538385,3225.259561,45.894517,229.794871,18.829294,53.178282,9692.279896,5168.809802,1.653283e+04,9669.374801,6687.534216,4777.369780,3940.769855,6.845714e+04,400.700809,44.236782,60.83051,42.698264,3829.523312,1024.033919,1.806684,5.953992,1.803841,1.805191,1.807133,1.806513
std,2226.900214,1130.650814,689.503957,153.053698,22.654295,801.833501,0.551702,2.551291,0.745426,8.870661,1.856124,9.409501,498.523223,468.583469,8.746035,21.538293,0.369989,1.184482,0.212698,0.404597,54.450619,29.989919,92.573627,54.282527,38.437133,27.017978,21.874851,389.790788,2.299218,0.345608,0.474671,0.331321,38.823869,26.638078,4250.325912,2.584675e+05,1.991711e+05,4459.499896,7747.852013,163.454052,682.402531,58.836650,174.463730,30613.410007,17788.666800,5.501074e+04,32842.826265,23604.353652,17622.230928,14743.984184,1.765861e+05,1129.555593,106.207249,133.43094,102.515843,8449.330908,3601.449915,2.673096,3.757249,0.472008,0.409653,0.202965,0.007516
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,34.000000,-327.870000,-15633.000000,-14885.000000,-342.000000,-1267.000000,-3.000000,-38.000000,-3.000000,-11.000000,-1577.000000,-1111.000000,-3184.000000,-1991.000000,-1548.000000,-1173.000000,-1104.000000,-14025.000000,-60.000000,-9.000000,-31.000000,-7.000000,-670.880000,-97.290000,-86.090000,-7.832000e+03,-7.205000e+03,-184.000000,-346.000000,-2.000000,-14.000000,-2.000000,-8.000000,-679.000000,-435.000000,-1.109000e+03,-755.000000,-628.000000,-460.000000,-439.000000,-5.339000e+03,-30.000000,-5.000000,-8.00000,-4.000000,-409.990000,0.000000,-8.537391,0.000000,0.000000,0.000000,0.000000,1.798679
25%,892.000000,313.000000,235.000000,43.000000,13.000000,631.000000,2.000000,15.000000,0.000000,18.000000,65.000000,0.200000,10.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [17]:
# train_data[train_data.isnull().any(axis=1)]

,ad_id,ad_set_id,campaign_id,product_id,account_id,post_id_emb,post_type,countries,gender,age_min,age_max,spend,impressions,reach,clicks,engagement_nums,post_shares,post_reactions,post_comments,post_saves,watch15s,watch30s,watch_p25,watch_p50,watch_p75,watch_p95,watch_p100,watches,add_cart_num,add_payment_info,initiates_checkout,purchase,income_1,income_2,cum_spend,cum_impressions,cum_reach,cum_clicks,cum_engagement_nums,cum_post_shares,cum_post_reactions,cum_post_comments,cum_post_saves,cum_watch15s,cum_watch30s,cum_watch_p25,cum_watch_p50,cum_watch_p75,cum_watch_p95,cum_watch_p100,cum_watches,cum_add_cart_num,cum_add_payment_info,cum_initiates_checkout,cum_purchase,cum_income_1,cum_income_2,roi,datetime_hour,product_id_roi_mean,account_id_roi_mean,countries_roi_mean,datetime_hour_roi_mean


In [ ]:
# random forest
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(train_data.iloc[:][feature_columns],
        train_data.iloc[:]['roi']
)
dev_data['rf_y_pred'] = model.predict(dev_data.iloc[:][feature_columns])
dev_data['rf_wmse'] = dev_data.apply(lambda x: (x['datetime_hour']+1) * 1.0 / 13 * abs(x['roi'] - x['rf_y_pred']), axis=1)
np.mean(dev_data['rf_wmse'])

In [ ]:
# 
import torch
torch.cuda.is_available()

In [ ]:
# Time Series

In [ ]:
# Linear Regression